As electronic vehicles (EVs) become more popular, there is an increasing need for access to charging stations, also known as ports. To that end, many modern apartment buildings have begun retrofitting their parking garages to include shared charging stations. A charging station is shared if it is accessible by anyone in the building.

<img src="charging_station.jpg" alt="EV Charging" width="500" height="auto">

But with increasing demand comes competition for these ports — nothing is more frustrating than coming home to find no charging stations available! In this project, you will use a dataset to help apartment building managers better understand their tenants’ EV charging habits.

The data has been loaded into a PostgreSQL database with a table named `charging_sessions` with the following columns:

## charging_sessions

| Column | Definition | Data type |
|-|-|-|
|`garage_id`| Identifier for the garage/building|`VARCHAR`|
|`user_id` | Identifier for the individual user|`VARCHAR`|
|`user_type`|Indicating whether the station is `Shared` or `Private`| `VARCHAR` |
|`start_plugin`|The date and time the session started |`DATETIME`|
|`start_plugin_hour`|The hour (in military time) that the session started | `NUMERIC`|
|`end_plugout`|The date and time the session ended | `DATETIME` |
|`end_plugout_hour`|The hour (in military time) that the session ended | `NUMERIC`|
|`duration_hours`| The length of the session, in hours|`NUMERIC`|
|`el_kwh`| Amount of electricity used (in Kilowatt hours)|`NUMERIC`|
|`month_plugin`| The month that the session started |`VARCHAR`|
|`weekdays_plugin`| The day of the week that the session started|`VARCHAR`|

Let’s get started!

#### Sources
- **Data**: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0), via [Kaggle](https://www.kaggle.com/datasets/anshtanwar/residential-ev-chargingfrom-apartment-buildings),
- **Image**: Julian Herzog, [CC BY 4.0](https://creativecommons.org/licenses/by/4.0), via Wikimedia Commons

Problem Statement:

As the popularity of electric vehicles (EVs) continues to grow, the demand for accessible charging stations in apartment buildings has increased. This rising demand can lead to competition for available charging ports, causing frustration among tenants. To address this issue, it is essential to understand the charging habits of tenants and the utilization patterns of the existing charging infrastructure. This analysis aims to provide apartment building managers with actionable insights into EV charging behavior, identify peak usage times, compare the usage of shared versus private stations, and highlight any patterns that could inform better management and expansion of charging facilities. By answering these key questions, we can help optimize the availability and efficiency of EV charging stations, ensuring a better experience for all residents.Let us start the analysis to understand EV charging habits of tenants in apartment buildings. 

Let us start the Analysis......

In [3]:
-- Return a tibble of the table charging_sessions to take a look

SELECT *
FROM charging_sessions
LIMIT 10;

,garage_id,user_id,user_type,start_plugin,start_plugin_hour,end_plugout,end_plugout_hour,el_kwh,duration_hours,month_plugin,weekdays_plugin
0,AdO3,AdO3-4,Private,2018-12-21 10:20:00+00:00,10,2018-12-21 10:23:00+00:00,10.0,0.30,0.050000,Dec,Friday
1,AdO3,AdO3-4,Private,2018-12-21 10:24:00+00:00,10,2018-12-21 10:32:00+00:00,10.0,0.87,0.136667,Dec,Friday
2,AdO3,AdO3-4,Private,2018-12-21 11:33:00+00:00,11,2018-12-21 19:46:00+00:00,19.0,29.87,8.216389,Dec,Friday
3,AdO3,AdO3-2,Private,2018-12-22 16:15:00+00:00,16,2018-12-23 16:40:00+00:00,16.0,15.56,24.419722,Dec,Saturday
4,AdO3,AdO3-2,Private,2018-12-24 22:03:00+00:00,22,2018-12-24 23:02:00+00:00,23.0,3.62,0.970556,Dec,Monday
5,AdO3,AdO3-2,Private,2018-12-24 23:32:00+00:00,23,2018-12-25 17:37:00+00:00,17.0,16.14,18.077778,Dec,Monday
6,AdO3,AdO3-2,Private,2018-12-25 18:25:00+00:00,18,2018-12-26 16:08:00+00:00,16.0,10.33,21.720833,Dec,Tuesday
7,AdO3,AdO3-4,Private,2018-12-26 10:41:00+00:00,10,2018-12-26 16:52:00+00:00,16.0,27.66,6.188056,Dec,Wednesday
8,AdO3,AdO3-2,Private,2018-12-26 18:46:00+00:00,18,2018-12-26 21:08:00+00:00,21.0,8.83,2.371111,Dec,Wednesday
9,AdO3,AdO3-2,Private,2018-12-29 16:04:00+00:00,16,2018-12-29 20:55:00+00:00,20.0,8.58,4.856111,Dec,Saturday


## 1. What are the peak hours for EV charging? 

In [10]:
-- Identifying peak hours helps in managing and possibly expanding charging infrastructure to accommodate high demand times.
SELECT start_plugin_hour, COUNT(*) AS total_sessions
FROM charging_sessions
GROUP BY start_plugin_hour
ORDER BY total_sessions DESC;
/* Looking at the total number of sessions by hour (0 to 23), most people try to charge between 15 (3 PM) to 19 (7 PM). */


,start_plugin_hour,total_sessions
0,16,966
1,15,787
2,19,676
3,17,608
4,18,584
5,20,583
6,21,463
7,14,408
8,22,374
9,13,294


## 2. What is the average duration of charging sessions?

In [18]:
-- Understanding the average duration can help in optimizing charging schedules and informing users about expected charging times.
SELECT AVG(duration_hours) AS avg_duration
FROM charging_sessions;
/* Average Duration of charging time is 11.5 Hours. */

,user_type,avg_duration
0,Shared,6.535401
1,Private,12.784593


## 3. How does the usage of shared vs. private stations compare along with number of unique users in each station ?

In [16]:
-- This comparison helps in determining if shared stations are sufficient or if more private stations are needed. Also, Number of Unique users based on station. 
SELECT user_type, 
       COUNT(*) AS total_sessions, 
       SUM(el_kwh) AS total_kwh,
       COUNT(DISTINCT user_id) AS unique_users
FROM charging_sessions
GROUP BY user_type;
/* Looking into this information seems like 

,user_type,total_sessions,total_kwh,unique_users
0,Private,5466,61231.37,56
1,Shared,1412,26261.91,52


## 4. What are the most common days for charging sessions?

In [21]:
-- Knowing the most common days can assist in planning maintenance and ensuring availability during high-demand days.
SELECT weekdays_plugin, COUNT(*) AS total_sessions
FROM charging_sessions
GROUP BY weekdays_plugin
ORDER BY total_sessions DESC;
/* Most common weekdays to charge EV vehicles are Thursday, Wednesday and Friday. */

,weekdays_plugin,total_sessions
0,Thursday,1072
1,Wednesday,1039
2,Friday,1034
3,Monday,981
4,Sunday,955
5,Tuesday,943
6,Saturday,854


## 5. How does charging activity vary by month?

In [39]:
-- Seasonal trends in charging can help in predicting future needs and planning for infrastructure upgrades.
SELECT month_plugin, COUNT(*) AS total_sessions, SUM(el_kwh) AS total_kwh
FROM charging_sessions
GROUP BY month_plugin
ORDER BY total_sessions DESC;
/* Months during the Fall highlight excessive need for the chaging stations. */

,month_plugin,total_sessions,total_kwh
0,Jan,1361,16783.46
1,Dec,1143,13823.54
2,Nov,1107,14091.95
3,Oct,845,10821.10
4,Sep,501,7507.08
5,Aug,362,5626.93
6,Apr,306,3175.91
7,Mar,301,3539.82
8,Jun,291,3819.44
9,May,288,3451.67


## 6. Which garages/buildings have the highest usage?

In [26]:
-- Identifying high-usage garages can highlight areas needing more charging stations or better management.
SELECT garage_id, COUNT(*) AS total_sessions, SUM(el_kwh) AS total_kwh
FROM charging_sessions
GROUP BY garage_id
ORDER BY total_sessions DESC;
/* Need to Investigate more about on BI2 garage */

,garage_id,total_sessions,total_kwh
0,Bl2,2243,27046.92
1,AsO2,665,10672.16
2,AdO1,586,8372.83
3,UT9,421,6789.82
4,AdO3,379,5239.22
5,UT7,313,2532.65
6,MS1,270,1763.17
7,AsO10,253,3428.27
8,NR1,224,1809.98
9,SR2,221,3768.99


## 7. What is the total electricity consumption for each garage/building?

In [40]:
-- Total electricity consumption data is crucial for energy management and planning utility needs.
SELECT garage_id, SUM(el_kwh) AS total_kwh
FROM charging_sessions
GROUP BY garage_id
ORDER BY total_kwh DESC;
/* Similar to what we have observed before; BI2 garage has the highest total electricity consumption. */

,garage_id,total_kwh
0,Bl2,27046.92
1,AsO2,10672.16
2,AdO1,8372.83
3,UT9,6789.82
4,AdO3,5239.22
5,SR14,3791.26
6,SR2,3768.99
7,AsO10,3428.27
8,MS22,2651.19
9,UT7,2532.65


## 8. What is the average electricity consumption per session?

In [29]:
-- Average consumption per session helps in understanding the efficiency and energy requirements of the charging sessions.
SELECT AVG(el_kwh) AS avg_kwh
FROM charging_sessions;
/* Average consumption per session is 12.72 kwh */

,avg_kwh
0,12.720744


## 9. What are the longest charging sessions?

In [41]:
-- Analyzing the longest sessions can reveal outliers and help in setting policies for fair usage.
SELECT *
FROM charging_sessions
ORDER BY duration_hours DESC;
/* This shows Outliers in the data, as end plugout time is not captured for 33 records */

,garage_id,user_id,user_type,start_plugin,start_plugin_hour,end_plugout,end_plugout_hour,el_kwh,duration_hours,month_plugin,weekdays_plugin
0,UT7,UT7-1,Private,2019-12-19 16:08:00+00:00,16,NaT,NaN,4.91,NaN,Dec,Thursday
1,MS22,MS22-1,Private,2019-12-30 20:56:00+00:00,20,NaT,NaN,18.61,NaN,Dec,Monday
2,SR2,Share-17,Shared,2019-12-23 11:23:00+00:00,11,NaT,NaN,0.05,NaN,Dec,Monday
3,SR2,Share-32,Shared,2019-12-23 11:23:00+00:00,11,NaT,NaN,3.55,NaN,Dec,Monday
4,MS22,MS22-1,Private,2019-12-16 15:18:00+00:00,15,NaT,NaN,22.73,NaN,Dec,Monday
...,...,...,...,...,...,...,...,...,...,...,...
6873,UT9,Bl2-7,Shared,2019-09-22 14:58:00+00:00,14,2019-09-22 14:58:00+00:00,14.0,0.01,0.005000,Sep,Sunday
6874,AsO2,AsO2-1,Private,2019-12-04 13:16:00+00:00,13,2019-12-04 13:16:00+00:00,13.0,0.01,0.004444,Dec,Wednesday
6875,NR1,NR1-3,Private,2019-10-10 07:35:00+00:00,7,2019-10-10 07:35:00+00:00,7.0,0.01,0.004167,Oct,Thursday
6876,UT7,UT7-3,Private,2019-10-10 16:41:00+00:00,16,2019-10-10 16:41:00+00:00,16.0,0.01,0.003611,Oct,Thursday


## 10. How many sessions start during each hour of the day?

In [43]:
--Understanding hourly session starts helps in identifying patterns and peak times for better management.
SELECT start_plugin_hour, COUNT(*) AS total_sessions
FROM charging_sessions
GROUP BY start_plugin_hour
ORDER BY total_sessions DESC;
/* Most charging stations are occupied around 4 PM, causing potential frustration due to unavailability. */


,start_plugin_hour,total_sessions
0,16,966
1,15,787
2,19,676
3,17,608
4,18,584
5,20,583
6,21,463
7,14,408
8,22,374
9,13,294


## 11. What is the distribution of session durations?

In [5]:
-- Distribution analysis of session durations helps in planning and optimizing station availability.
SELECT duration_hours, COUNT(*) AS total_sessions
FROM charging_sessions
GROUP BY duration_hours
ORDER BY duration_hours;
/* This Analysis will be helpful in understanding teh trends when projected in the form of a Histogram */

,duration_hours,total_sessions
0,0.003333,1
1,0.003611,1
2,0.004167,1
3,0.004444,1
4,0.005000,1
...,...,...
6497,235.020278,1
6498,237.033333,1
6499,239.240278,1
6500,255.029444,1


## 12. Are there any significant patterns in charging behavior over the week?

In [46]:
SELECT weekdays_plugin, start_plugin_hour, COUNT(*) AS total_sessions
FROM charging_sessions
GROUP BY weekdays_plugin, start_plugin_hour
ORDER BY weekdays_plugin, start_plugin_hour;


,weekdays_plugin,start_plugin_hour,total_sessions
0,Friday,0,11
1,Friday,1,4
2,Friday,2,3
3,Friday,3,1
4,Friday,4,2
...,...,...,...
158,Wednesday,19,105
159,Wednesday,20,113
160,Wednesday,21,78
161,Wednesday,22,66


## 13. What are the top users based on electricity consumption?

In [36]:
SELECT user_id, SUM(el_kwh) AS total_kwh
FROM charging_sessions
GROUP BY user_id
ORDER BY total_kwh DESC
LIMIT 10;

,user_id,total_kwh
0,Bl2-5,3696.16
1,AdO1-3,3587.44
2,Bl2-2,3288.96
3,AsO6-1,3043.23
4,SR14-2,2943.57
5,SR2-3,2922.53
6,Share-12,2743.50
7,Bl2-3,2737.72
8,AsO2-1,2644.69
9,Bl2-1,2579.38


## 14. How does charging activity differ between weekdays and weekends?

In [38]:
-- Differentiating between weekday and weekend activity helps in understanding how usage patterns shift and planning accordingly.
SELECT CASE 
           WHEN weekdays_plugin IN ('Saturday', 'Sunday') THEN 'Weekend'
           ELSE 'Weekday'
       END AS day_type, COUNT(*) AS total_sessions, SUM(el_kwh) AS total_kwh
FROM charging_sessions
GROUP BY day_type;
/* This doesn't give much Info */

,day_type,total_sessions,total_kwh
0,Weekend,1809,24815.10
1,Weekday,5069,62678.18


## 15. What are the number of unique users for garage?

In [3]:
-- unique_users_per_garage
-- This query finds the number of unique users per garage.
SELECT garage_id, COUNT(DISTINCT user_id) AS num_unique_users
FROM charging_sessions
WHERE user_type = 'Shared'
GROUP BY garage_id
ORDER BY num_unique_users DESC;
/* BI2 has the highest number of Unique Users */ 

,garage_id,num_unique_users
0,Bl2,18
1,AsO2,17
2,UT9,16
3,AdO3,3
4,MS1,2
5,SR2,2
6,AdA1,1
7,Ris,1


## Findings
1. **Peak Hours for EV Charging:** The busiest hours for charging sessions are between 3 PM and 7 PM, with the highest number of sessions starting at 4 PM.
2. **Average Duration of Charging Sessions:** The overall average duration of charging sessions is approximately 11.5 hours. Shared stations have shorter average sessions (around 6.5 hours), while private stations have longer sessions (approximately 12.8 hours).
3. **Usage of Shared vs. Private Stations:** 
- Private stations are used more frequently than shared stations, with 5466 sessions compared to 1412 sessions for shared stations.
- Private stations consume more electricity (61231.37 kWh) compared to shared stations (26261.91 kWh).
- The number of unique users for private stations is slightly higher (56) compared to shared stations (52).
4. **Most Common Days for Charging Sessions:** The highest charging activity occurs on Thursdays, Wednesdays, and Fridays.
5. **Monthly Charging Activity:** The fall months (October to December) show the highest charging activity, with December and January being particularly busy.
6. **High-Usage Garages:** The garage with the highest usage is BI2, with 2243 sessions and the highest total electricity consumption (27046.92 kWh).
7. **Electricity Consumption per Session:** The average electricity consumption per session is 12.72 kWh.
8. **Longest Charging Sessions:** Some charging sessions are significantly long, with certain sessions missing end plug-out times, indicating potential outliers or data recording issues.
9. ** Hourly Charging Patterns:** Most charging sessions start around 4 PM, indicating a peak usage time that may lead to frustration due to unavailability.
10. **Top Users Based on Electricity Consumption:** The top users, such as user Bl2-5, have significantly high electricity consumption, with the top 10 users consuming between 2579.38 kWh and 3696.16 kWh.
11. **Weekday vs. Weekend Charging Activity:** Weekdays see higher charging activity (5069 sessions and 62678.18 kWh) compared to weekends (1809 sessions and 24815.1 kWh).
12. **Unique Users per Garage:** The garage BI2 has the highest number of unique users (18) for shared stations, followed by AsO2 (17) and UT9 (16).

## Recommendations

1. **Optimize Charging Infrastructure:**
- Increase the number of charging stations in high-usage garages, particularly BI2, to alleviate congestion during peak hours.
- Consider expanding shared station availability to better accommodate demand, especially during the afternoon peak hours.

2. **Manage Peak Usage Times:**
- Implement a reservation system to allow tenants to book charging slots, especially during peak hours (3 PM to 7 PM).
- Encourage off-peak charging by providing incentives or discounted rates for charging sessions outside the peak hours.

3. **Enhance Data Collection:**
- Address the data recording issues for end plug-out times to ensure accurate tracking of session durations.
- Regularly monitor and clean the data to identify and rectify any outliers or inconsistencies.

4. **Promote Efficient Charging Habits:**
- Educate tenants on optimal charging times and the benefits of shorter charging sessions to improve station availability.
- Implement policies or guidelines to limit excessively long charging sessions, ensuring fair usage for all residents.

5. **Plan for Future Expansion:**
- Based on the increasing trend of EV adoption and the high demand during certain months, plan for future expansions of the charging infrastructure to accommodate growing needs.
- Evaluate the potential for installing additional charging stations in garages with high unique user counts and electricity consumption.

## Conclusion
The analysis of EV charging habits in apartment buildings reveals significant insights into peak usage times, average session durations, and the comparative usage of shared versus private stations. High-demand periods, particularly in the afternoons and specific fall months, highlight the need for optimized charging infrastructure and better management strategies. By implementing the recommendations, apartment building managers can enhance the availability and efficiency of EV charging stations, ultimately providing a better experience for tenants and supporting the growing adoption of electric vehicles.
